AUTHORS: **ÓSCAR PIETTE AND LUCÍA SÁNCHEZ**


# **Pneumonia identification from X-Ray images**:

The aim of this project is to construct a classifier that allows us to differ patients with pneumonia from normal patients employing X-Ray images. To do so, we used transfer learning by employing the pre-trained Keras deep learning model **ResNet50** as a base and modifying it in order to adapt it to our data. We also used the library Orca from Analytics Zoo which allows process distributed Big Data. All the dataset is in https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia

Firstly, we start by importing all the necessary libraries and dependencies:

**Intalling Analytics Zoo:**

In [1]:
# Install jdk8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
# Set environment variable JAVA_HOME.
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version

openjdk version "1.8.0_312"
OpenJDK Runtime Environment (build 1.8.0_312-8u312-b07-0ubuntu1~18.04-b07)
OpenJDK 64-Bit Server VM (build 25.312-b07, mixed mode)


In [2]:
# Install latest pre-release version of Analytics Zoo
# Installing Analytics Zoo from pip will automatically install pyspark, bigdl, and their dependencies.
!pip install --pre --upgrade analytics-zoo[ray]
exit() # refresh the pkg you just install

**Installing tensorflow:**

In [3]:
!pip install tensorflow==2.7.0
import tensorflow as tf
tf.__version__

'2.7.0'

**Initialize the Orca context:**

In [ ]:
from zoo.orca import init_orca_context, stop_orca_context
from zoo.orca import OrcaContext

# It is recommended to set it to True when running Analytics Zoo in Jupyter notebook 
OrcaContext.log_output = True 

init_orca_context(cluster_mode="local", cores=2)

In [5]:
#Obtain the spark session from the Orca Context
spark = OrcaContext.get_spark_session()

The data used for this project is located in kaggle. One option is to download the data from drive and upload it to drive to further use it in collab, but the option used was to get the data directly from kaggle. To do so we follow the instructions indicated in this source: https://buggyprogrammer.com/load-kaggle-dataset-in-colab-or-jupyter/

In [6]:
!pip install kaggle

In [7]:
#Make a directory named kaggle and copy the kaggle.json file there:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# change the permission of the file
!chmod 600 ~/.kaggle/kaggle.json

In [8]:
# Obtain the dataset:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

chest-xray-pneumonia.zip: Skipping, found more recently modified local copy (use --force to force download)


In [9]:
# Unziping the dataset:
from zipfile import ZipFile
file_name = 'chest-xray-pneumonia.zip' 
with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [10]:
# Establish the path to each folder:
test_path="/content/chest_xray/test/"
train_path="/content/chest_xray/train/"
val_path="/content/chest_xray/val/"

The dataset it is composed by three folders: Train, Test and Validation. Each of them has two folders, one for the "pneumonia" images and other for the "normal" images. There are 16 images in the validation folder, 624 images in the test folder and 5216 images in the train folder. 

In order to manage the data, we used Keras and TensorFlow which we can use in the Orca context. Keras is a neural network library and TensorFlow is a open-source library for a number of various tasks in machine learning. They were used to prepare and preprocess the data; and to construct our model using transfer learning. 

In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
# importing the necessary libraries

Following the indications in https://analytics-zoo.readthedocs.io/en/latest/doc/Orca/QuickStart/orca-tf2keras-quickstart.html, we constructed three functions, one for the training dataset, one for the test dataset and another for the validation dataset. With the function "image_dataset_from_directory" we obtained a tensorflow dataset with the images divided by the labels "NORMAL" and "PNEUMONIA". We also established that the images size should be 224 x 224 (which it is optimal for ResNet50). 

In the training function we perfom a data augmentation process and a duplication of the dataset in order to increase the number of images because it is really small for a deep learning process. We also perfom a duplication event in the validation dataset. For the three datasets, we perfom a general preprocess event with the function preprocess_input. 
 


In [12]:
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications.resnet50 import preprocess_input

# Obtain and preprocess the training dataset:
def train_data_creator(config, batch_size):
    
    # Obtaining the data from the directory:
    data_train = image_dataset_from_directory(train_path, 
                                            image_size=(224, 224), #Establish a general size for all images as 224 x 224
                                            batch_size = 1,
                                            shuffle=False,
                                            label_mode='binary',
                                            class_names = ['NORMAL', 'PNEUMONIA'])
   
    ##### Perform data augmentation:
    # It randomly flips the images horizontally
    # It randomly rotates the images by rotating counter 0.15 * 2pi radians clock-wise 
    # It randomly adjusts the contrast of the images by a factor of 0.1. Contrast is adjusted independently for each channel of each image during training
    # It randomly translates the images, it shift the image 0.1 down vertically and horizontally
    data_augmentation = keras.Sequential(
        [
            layers.RandomRotation(factor=0.15),
            layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
            layers.RandomFlip(),
            layers.RandomContrast(factor=0.1)
        ]
    )
    # Duplicate the data:
    data_train = data_train.repeat(2)

    # Applying the data augmentation proccess:
    data_train = data_train.map(lambda x, y: (data_augmentation(x, training = True), y))

    # Applying a preprocessing function from Keras to the images:
    data_train = data_train.map(lambda x, y: (keras.applications.resnet50.preprocess_input(x), y))
    
    # Transform the data into tf.float32:
    data_train = data_train.map(lambda x, y: (tf.cast(x, dtype = tf.float32), y))

    return data_train

# Obtain and preprocess the test dataset:
def test_data_creator(config, batch_size):
    
    # Obtaining the data from the directory:
    data_test = image_dataset_from_directory(test_path, 
                                            image_size=(224, 224), 
                                            batch_size = batch_size,
                                            shuffle=True,
                                            label_mode='binary',
                                            class_names = ['NORMAL', 'PNEUMONIA'])
   
    # Applying a preprocessing function from Keras to the images:
    data_test = data_test.map(lambda x, y: (keras.applications.resnet50.preprocess_input(x), y))
    
    # Transform the data into tf.float32:
    data_test = data_test.map(lambda x, y: (tf.cast(x, dtype = tf.float32), y))
    
    return data_test

# Obtain and preprocess the validation dataset:
def val_data_creator(config, batch_size):
    
    # Obtaining the data from the directory:
    data_val = image_dataset_from_directory(val_path, 
                                            image_size=(224, 224), 
                                            batch_size = 1,
                                            shuffle=False,
                                            label_mode='binary',
                                            class_names = ['NORMAL', 'PNEUMONIA'])
    # Duplicate the data:
    data_val = data_val.repeat(2)

    # Applying a preprocessing function from Keras to the images:
    data_val = data_val.map(lambda x, y: (keras.applications.resnet50.preprocess_input(x), y))
    # Casting it to tf.float32 to make conversions easier (because it also gave problems without it)
    data_val = data_val.map(lambda x, y: (tf.cast(x, dtype = tf.float32), y))
    
    # Returning the processed dataset
    return data_val



In order to use the model in the orca context, we first need to define a model creation function, which will be used to import a model created in keras. The model_creation function serves that purpose and imports the already existing ResNet50 model and modifies it in keras before compiling it. This process allows for the usage of this model in an Orca context through an Estimator created with the function from_keras.

In [13]:
def model_creation(config):
  
  from tensorflow.keras.applications import ResNet50
  from tensorflow.keras.models import Sequential
  from tensorflow.keras.layers import Dense

  base_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")
  for layer in base_model.layers:
    layer.trainable = False

  base_model = Sequential()
  base_model.add(ResNet50(include_top=False, weights='imagenet', pooling='max'))
  base_model.add(Dense(1, activation='sigmoid'))
  base_model.compile(optimizer = tf.keras.optimizers.SGD(lr=0.0001), loss = 'binary_crossentropy', metrics = ['acc'])

  return base_model


In [14]:
# Defining a Keras callback to use early stopping inside of the Orca estimator
my_callback = tf.keras.callbacks.EarlyStopping(
   monitor="val_loss",
  patience=2,
  verbose=2,
  restore_best_weights=True,
)

In [15]:
from zoo.orca.learn.tf2.estimator import Estimator
# using Orca to create an Estimator from the custom model created by the model_creator function
est = Estimator.from_keras(model_creator=model_creation)

2022-02-10 20:04:59,798	INFO services.py:1174 -- View the Ray dashboard at http://172.28.0.2:8265


{'node_ip_address': '172.28.0.2', 'raylet_ip_address': '172.28.0.2', 'redis_address': '172.28.0.2:6379', 'object_store_address': '/tmp/ray/session_2022-02-10_20-04-59_120376_2403/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-02-10_20-04-59_120376_2403/sockets/raylet', 'webui_url': '172.28.0.2:8265', 'session_dir': '/tmp/ray/session_2022-02-10_20-04-59_120376_2403', 'metrics_export_port': 57393, 'node_id': 'fd5560ec20d961218b3c02469e64faffd1ec2065a5be9123ce331998'}


(pid=2681) WARNING:tensorflow:From /usr/local/lib/python3.7/dist-packages/zoo/orca/learn/tf2/tf_runner.py:317: _CollectiveAllReduceStrategyExperimental.__init__ (from tensorflow.python.distribute.collective_all_reduce_strategy) is deprecated and will be removed in a future version.
(pid=2681) Instructions for updating:
(pid=2681) use distribute.MultiWorkerMirroredStrategy instead
(pid=2681) 2022-02-10 20:05:06.301000: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Finally, the last steps are to train the model and to evaluate its results:

In [ ]:
stats = est.fit(train_data_creator,
                epochs=1,
                batch_size = 32,
                validation_data=val_data_creator)
                


(pid=2681) Found 5216 files belonging to 2 classes.


(pid=2681) WARNING:tensorflow:AutoGraph could not transform <function train_data_creator.<locals>.<lambda> at 0x7fe0c84394d0> and will run it as-is.
(pid=2681) Cause: could not parse the source code of <function train_data_creator.<locals>.<lambda> at 0x7fe0c84394d0>: no matching AST found among candidates:
(pid=2681) 
(pid=2681) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
(pid=2681) WARNING:tensorflow:AutoGraph could not transform <function train_data_creator.<locals>.<lambda> at 0x7fe0c8439dd0> and will run it as-is.
(pid=2681) Cause: could not parse the source code of <function train_data_creator.<locals>.<lambda> at 0x7fe0c8439dd0>: no matching AST found among candidates:
(pid=2681) 
(pid=2681) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
(pid=2681) WARNING:tensorflow:AutoGraph could not transform <function train_data_creator.<locals>.<lambda> at 0x7fe0c8439d40> and will run it as-is.

(pid=2681) Found 16 files belonging to 2 classes.


(pid=2681) 2022-02-10 20:05:16.160341: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_1"
(pid=2681) op: "TensorSliceDataset"
(pid=2681) input: "Placeholder/_0"
(pid=2681) attr {
(pid=2681)   key: "Toutput_types"
(pid=2681)   value {
(pid=2681)     list {
(pid=2681)       type: DT_STRING
(pid=2681)     }
(pid=2681)   }
(pid=2681) }
(pid=2681) attr {
(pid=2681)   key: "_cardinality"
(pid=2681)   value {
(pid=2681)     i: 5216
(pid=2681)   }
(pid=2681) }
(pid=2681) attr {
(pid=2681)   key: "is_files"
(pid=2681)   value {
(pid=2681)     b: false
(pid=2681)   }
(pid=2681) }
(pid=2681) attr {
(pid=2681)   key: "metadata"
(pid=2681)   value {
(pid=2681)     s: "\n\024TensorSliceDataset:0"
(pid=2681)   }
(pid=2681) }
(pid=2681) attr {
(pid=2681)   key: "output_shapes"
(pid=2681)   value {


  371/10432 [>.............................]
(pid=2681)  - ETA: 2:26:47 - loss: 0.0286 - acc: 0.9919
  464/10432 [>.............................]
(pid=2681)  - ETA: 2:24:28 - loss: 0.0229 - acc: 0.9935
  507/10432 [>.............................]
(pid=2681)  - ETA: 2:23:36 - loss: 0.0210 - acc: 0.9941
  548/10432 [>.............................]
(pid=2681)  - ETA: 2:22:56 - loss: 0.0194 - acc: 0.9945
  614/10432 [>.............................] - ETA: 2:21:52 - loss: 0.0173 - acc: 0.9951
(pid=2681) 
(pid=2681)   615/10432 [>.............................] - ETA: 2:21:51 - loss: 0.0173 - acc: 0.9951
  675/10432 [>.............................]
(pid=2681)  - ETA: 2:22:11 - loss: 0.0158 - acc: 0.9956
  685/10432 [>.............................]
(pid=2681)  - ETA: 2:21:59 - loss: 0.0156 - acc: 0.9956
  718/10432 [=>............................]
(pid=2681)  - ETA: 2:22:14 - loss: 0.0148 - acc: 0.9958
 122

In [ ]:
val_stats = est.evaluate(val_data_creator, batch_size = 32)
val_stats

In [ ]:
# Evaluation with the test set
test_stats = est.evaluate(test_data_creator, batch_size = 32)
test_stats

In [ ]:
for stat, value in test_stats.items():
  print (stat, ": ", value, sep = "")